In [83]:
import numpy as np
import pandas as pd
from collections import defaultdict
import pickle

In [6]:
test_data = pd.read_csv("test_batch0.csv")
test_data.head()

,Unnamed: 0,identifier,coauthors,year,abstract,venue,title,input
0,0,0,"[16336, 1762, 4357, 12564]",19,"[37, 1662, 3207, 10, 33, 2037, 1738, 1642, 155...",223,"[3207, 24, 1798, 1738, 37, 2375, 1568, 11, 53,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,1,"[21189, 14088]",19,"[1731, 2130, 3674, 1705, 1656, 3077, 1546, 367...",223,"[40, 1560, 1536, 1544, 1609, 1705, 1658, 1543,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,2,2,"[3625, 1198, 19889, 794, 2749, 7801]",19,"[1551, 1728, 3920, 1542, 1535, 1656, 1543, 153...",7,"[47, 1574, 1729, 1641, 11, 37, 2533, 2015, 47,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,3,"[19810, 15173, 5876, 111]",19,"[51, 1535, 2115, 1543, 1811, 1700, 1657, 1684,...",21,"[1770, 53, 2054, 1549, 1529, 1723, 2796, 1547,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,4,"[10932, 7668, 11907, 19601, 15307, 10492, 1049...",19,"[1775, 1746, 1842, 1525, 33, 2551, 1882, 1542,...",-1,"[18, 1924, 23, 1544, 3927, 2686, 1543, 1535, 1...","[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 4, ..."


In [7]:
for i in range(1, 8):
    next_batch = pd.read_csv("test_batch" + str(i) + ".csv")
    test_data = pd.concat([test_data, next_batch])

In [10]:
test_data.shape

(800, 8)

In [84]:
with open('pro_venue_dict.pickle', 'rb') as handle:
    pro_venue_dict = pickle.load(handle)

In [85]:
pro_venue_dict[23]

{144: 1,
 21: 24,
 89: 1,
 59: 1,
 203: 1,
 206: 1,
 0: 2,
 '': 2,
 9: 1,
 30: 1,
 91: 1,
 20: 1,
 142: 1}

Start predicitons

In [225]:
import json
def past_collabs(row):
    input_list = json.loads(row["input"])
    return input_list[:100]

def abstract_similarities(row):
    input_list = json.loads(row["input"])
    return input_list[100:]

def ok_venue(row):
    pass

def predict(test_row):
    collab_list = np.asarray(past_collabs(test_row)) # past collaborations list
    collabs = np.where(collab_list > 0)[0] # past collaborators 
    high_collabs = np.where(collab_list > 1)[0] # past collaborators collaborated more than once
    very_high_collabs = np.where(collab_list >= 4)[0] # past collaborators collaborated at least four time
    
    simil = np.asarray(abstract_similarities(test_row))
    high_simil_authors = np.where(simil > 0.4)[0]
    high_simil = simil[high_simil_authors]
#     print("===========")
#     print("past collab")
#     print(collabs)
#     print("===========")
#     print("high sim")
#     print(high_simil_authors)
#     print(high_simil)
#     print("===========")
 
################## Oct 15 version 
    prediction = []
    if len(collabs) == 0:
        prediction = high_simil_authors
        
    elif len(collabs) >= 3:
        more_collab_authors = np.where(collab_list > 1)[0]
        if len(more_collab_authors) < 3:
            prediction = more_collab_authors
            return prediction
        else:
            sorted_more_collab_authors = sorted(more_collab_authors, key = lambda x : simil[x], reverse=True)
            prediction = sorted_more_collab_authors[:3]
            return prediction
    else:
        prediction = collabs
################## 
    
#     prediction = []
#     '''
#     stage one, focus on highly collaborated authors
#     '''
#     first_round = []
#     for past_collab in very_high_collabs:
#         first_round.append(past_collab)
    
#     # still too many authors? use venue existence
#     if len(first_round) <= 2:
#         prediction = first_round
#     else:
#         second_round = []
#         for pred in first_round:
#             if test_row["venue"] in pro_venue_dict[pred].keys() or test_row["venue"] == -1:
#                 second_round.append(pred)
        
#         # second round still too many authors? use venue counts
#         if len(second_round) <= 2:
#             prediction = second_round
#         else:
#             third_round = []
#             for pred in first_round:
#                 if test_row["venue"] in pro_venue_dict[pred].keys() and pro_venue_dict[pred][test_row["venue"]] > 2:
#                     third_round.append(pred)
#             prediction = third_round
        
    
#     if len(prediction) <= 2 and len(prediction) > 0:
#         return prediction
    
#     # at below stage, either prediction contains very high collab authors, but too many.. (100 - 200 cases?)
#     # or there's no very_high_collab authors
#     '''
#     stage two, focus on previously collaborated authors. 
#     ''' 
#     prediction = []
#     first_round = []
#     for past_collab in collabs:
#         first_round.append(past_collab)
#     # still too many authors? use venue existence
#     if len(first_round) <= 2:
#         prediction = first_round
#     else:
#         second_round = []
#         for pred in first_round:
#             if test_row["venue"] in pro_venue_dict[pred].keys() or test_row["venue"] == -1:
#                 second_round.append(pred)
        
#         # second round still too many authors? use venue counts
#         if len(second_round) <= 2:
#             prediction = second_round
#         else:
#             third_round = []
#             for pred in first_round:
#                 if test_row["venue"] in pro_venue_dict[pred].keys() and pro_venue_dict[pred][test_row["venue"]] > 2:
#                     third_round.append(pred)
#             prediction = third_round
            
#     if len(prediction) <= 2 and len(prediction) > 0:
#         return prediction
    
#     # at below stage, possibly prediction contains too many collab authors (118 cases)
#     # but mostly there's no previously collaborated authors (201 cases)
    
#     # in here use similarity > 0.4 get doen from 308 to 271
#     '''
#     stage three, focus on not previously collaborated authors. 
#     ''' 
#     prediction = []
    
#     for simi_author in high_simil_authors: 
#         prediction.append(simi_author)
        
#     if len(prediction) <= 2 and len(prediction) > 0:
#         return prediction
    
#     '''
#     stage four, check venue again? (venue + simil)
#     ''' 
#     prediction = []
#     anal.append(test_row["venue"]) # for anal
    
#     scores = [0] * 100
#     for pro_author in range(100):
#         if pro_author in pro_venue_dict.keys() and test_row["venue"] in pro_venue_dict[pro_author].keys():
#             scores[pro_author] += pro_venue_dict[pro_author][test_row["venue"]]
            
#     print(scores)
    
    return prediction

In [ ]:
def predict_v2(test_row):
    collab_list = np.asarray(past_collabs(test_row)) # past collaborations list
    collabs = np.where(collab_list > 0)[0] # past collaborators 
    high_collabs = np.where(collab_list > 1)[0] # past collaborators collaborated more than once
    very_high_collabs = np.where(collab_list >= 4)[0] # past collaborators collaborated at least four time
    
    simil = np.asarray(abstract_similarities(test_row))
    high_simil_authors = np.where(simil > 0.4)[0]
    high_simil = simil[high_simil_authors]
    ok_simil_authors = np.where(simil > 0.2)[0]
    
    
    
    

In [226]:
# c = 0
all_predictions = []
for i, row in test_data.iterrows():
    prediction = predict(row)
    all_predictions.append(prediction)
#     print(prediction)
#     c += 1
#     if c > 10:
#         break

In [227]:
all_predictions

[array([92]),
 array([84]),
 array([31]),
 array([23]),
 [59, 14, 53],
 array([74]),
 array([37]),
 [79, 60, 6],
 array([], dtype=int64),
 array([51]),
 [38, 79, 60],
 array([36, 42]),
 array([26, 47, 81, 95]),
 [13, 71, 56],
 [12, 77, 94],
 [9, 30, 82],
 array([84]),
 [49, 20, 43],
 array([37]),
 [43, 38, 60],
 [83, 97, 20],
 array([43, 56]),
 array([], dtype=int64),
 array([35]),
 array([50]),
 [98, 77, 68],
 array([35]),
 [38, 91, 60],
 array([], dtype=int64),
 array([ 0, 82]),
 array([], dtype=int64),
 [12, 20, 3],
 array([52, 65]),
 array([84]),
 array([59, 69]),
 array([81]),
 array([], dtype=int64),
 array([], dtype=int64),
 array([32, 98]),
 array([], dtype=int64),
 array([10, 58]),
 array([75]),
 [12, 20, 3],
 array([6]),
 array([], dtype=int64),
 array([54]),
 array([26, 81]),
 array([10, 92]),
 array([ 6, 73]),
 array([], dtype=int64),
 array([42]),
 array([38]),
 array([47]),
 [79, 23, 71],
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([5

In [230]:
with open('lr_pred.pickle', 'rb') as handle:
    lr_pred = pickle.load(handle)
lr_pred

[[92],
 [],
 [31],
 [23],
 [32],
 [],
 [],
 [6, 13],
 [],
 [],
 [38],
 [42],
 [],
 [13, 71],
 [94],
 [82],
 [84],
 [20],
 [],
 [38],
 [83],
 [43],
 [],
 [35],
 [50],
 [98],
 [35],
 [38],
 [],
 [],
 [],
 [12],
 [65],
 [84],
 [],
 [81],
 [],
 [],
 [98],
 [],
 [],
 [],
 [12],
 [],
 [],
 [],
 [26],
 [10],
 [6],
 [],
 [42],
 [38],
 [],
 [],
 [],
 [],
 [],
 [51],
 [],
 [29],
 [],
 [],
 [66],
 [],
 [],
 [],
 [10],
 [30],
 [],
 [59],
 [16],
 [85],
 [],
 [],
 [],
 [80],
 [],
 [],
 [],
 [],
 [],
 [20],
 [10],
 [],
 [41],
 [],
 [],
 [],
 [],
 [],
 [37],
 [],
 [],
 [],
 [],
 [],
 [67],
 [73],
 [82],
 [92],
 [],
 [],
 [6],
 [77],
 [],
 [75],
 [],
 [],
 [38],
 [],
 [],
 [],
 [],
 [],
 [],
 [25],
 [],
 [0],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [3],
 [47, 96],
 [94],
 [5],
 [89],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [25],
 [],
 [],
 [],
 [89],
 [],
 [38, 91],
 [],
 [62],
 [],
 [],
 [],
 [],
 [],
 [],
 [84],
 [],
 [],
 [72],
 [69],
 [],
 [82],
 [],
 [45],
 [],
 [78],
 [5],
 [],
 [],
 [37],
 [],
 [2,

In [231]:
for i in range(800):
    for k in all_predictions[i]:
        if k not in lr_pred[i]:
            lr_pred[i].append(k)

In [232]:
lr_pred

[[92],
 [84],
 [31],
 [23],
 [32, 59, 14, 53],
 [74],
 [37],
 [6, 13, 79, 60],
 [],
 [51],
 [38, 79, 60],
 [42, 36],
 [26, 47, 81, 95],
 [13, 71, 56],
 [94, 12, 77],
 [82, 9, 30],
 [84],
 [20, 49, 43],
 [37],
 [38, 43, 60],
 [83, 97, 20],
 [43, 56],
 [],
 [35],
 [50],
 [98, 77, 68],
 [35],
 [38, 91, 60],
 [],
 [0, 82],
 [],
 [12, 20, 3],
 [65, 52],
 [84],
 [59, 69],
 [81],
 [],
 [],
 [98, 32],
 [],
 [10, 58],
 [75],
 [12, 20, 3],
 [6],
 [],
 [54],
 [26, 81],
 [10, 92],
 [6, 73],
 [],
 [42],
 [38],
 [47],
 [79, 23, 71],
 [],
 [],
 [],
 [51, 84],
 [],
 [29],
 [43, 75],
 [],
 [66],
 [37],
 [31, 95],
 [],
 [10],
 [30, 82, 9],
 [8, 42],
 [59],
 [16],
 [85, 96],
 [56],
 [62],
 [6],
 [80],
 [12, 77],
 [68, 86],
 [82],
 [44],
 [],
 [20, 18],
 [10],
 [],
 [41, 37],
 [37, 60],
 [],
 [],
 [],
 [52, 65],
 [37, 12, 77],
 [],
 [],
 [63],
 [6, 71, 79],
 [74],
 [67],
 [73, 71],
 [82, 9, 30],
 [92, 10, 54],
 [87],
 [75],
 [6, 73, 71],
 [77, 12, 18],
 [65],
 [75, 5],
 [],
 [],
 [38],
 [62],
 [],
 [],
 [

Save to csv

In [234]:
def list_of_integers_to_string_format(integers):
    """Convert integer author ids to string representation

    Parameters
    ----------        
    list of integers : e.g. [1, 2]
        
    -------
    Return: string of format e.g. "1 2"
    
    """
    return " ".join([str(author_id) for author_id in integers])

def integers_to_csv(test_integers, test_ids):
    string_integers = list(map(list_of_integers_to_string_format, test_integers))
    
    header = ["ID", "Predict"]
    test_ids = [str(test_id) for test_id in test_ids]
    results = list(zip(test_ids, string_integers))
    output = [header] + results
    a = np.asarray(output)
    np.savetxt("combined_lr.csv", a, delimiter=",", fmt='%s')

In [235]:
integers_to_csv(lr_pred, test_data["identifier"].tolist())

Diagnostics

In [214]:
def sanity_check(all_predictions):
    count = 0
    for pred in all_predictions:
        if len(pred) == 0:
            count += 1
    print("# empty predictions: " + str(count))

In [217]:
sanity_check(all_predictions)

# empty predictions: 218


In [216]:
anal = []

In [199]:
Counter(anal)

Counter({-1: 48,
         223: 74,
         463: 1,
         7: 21,
         458: 8,
         459: 4,
         21: 10,
         59: 10,
         0: 50,
         197: 2,
         460: 4,
         105: 3,
         31: 3,
         461: 4,
         431: 2,
         63: 3,
         122: 1,
         106: 2,
         18: 4,
         235: 7,
         462: 1,
         234: 1,
         5: 1,
         13: 4,
         447: 2,
         17: 1})

Test data analysis

In [107]:
from collections import Counter

In [108]:
Counter(test_data["venue"].values)

Counter({223: 164,
         7: 117,
         21: 41,
         -1: 91,
         0: 187,
         463: 3,
         106: 4,
         367: 6,
         235: 15,
         56: 3,
         462: 5,
         458: 9,
         459: 8,
         63: 5,
         59: 29,
         431: 20,
         17: 6,
         197: 5,
         31: 16,
         460: 6,
         105: 8,
         13: 11,
         5: 7,
         447: 5,
         234: 4,
         464: 2,
         455: 1,
         461: 5,
         122: 4,
         161: 1,
         18: 9,
         263: 2,
         224: 1})

In [222]:
num_coauthors = [len(coauthors) for coauthors in test_data["coauthors"]]
Counter(num_coauthors)

Counter({26: 2, 34: 1, 19: 1, 31: 1, 18: 1, 6: 1, 20: 1})

In [223]:
for i in test_data["coauthors"]:
    print(i)
    break

[16336, 1762, 4357, 12564]


House keeping

In [224]:
past_collabs(test_row)

NameError: name 'test_row' is not defined